# MAT258a
Kevin Mallon
Homework 5
Problem 4

## Results
https://docs.google.com/spreadsheets/d/1gaGwP-BuMoko4a-6pwfsBzsuA_ukOFhTFzWCooKMUvI/edit?usp=sharing

## Info
Implementation of an augmented Lagrangian algorithm using BFGS for minimization. The BFGS method uses Lp and the gradient of Lp to approximate the Hessian of Lp. A linesearch meeting both Wolfe conditions is used to ensure sufficient descent in a converging direction. The augmented Lagrangian algorith itself checks if the norm of the constraints c(x) is less than some eta. If so, it recomputes y and shrinks eta. If not, it increases the penalty, rho.



In [9]:
function hockobj(x, pnum)
    x1=x[1]
    x2=x[2]
    if pnum==6
        fx=(1-x1)^2
        gx=[2x1-2;0]
        Hx=[2 0;0 0]
        cx=10*(x2-x1^2)
        Jx=[-20x1 20] 
    elseif pnum==7
        fx=log(1+x1^2)-x2
        gx=[2x1/(1+x1^2); -1]
        Hx=[-2(x1^2-1)/(x1^2+1)^2 0;0 0]
        cx=(1+x1^2)^2+x2^2-4
        Jx=[4(x1^3+x1) 2x2] 
    elseif pnum==8
        fx=-1
        gx=[0;0]
        Hx=[0 0;0 0]
        cx=[x1^2+x2^2-25;x1*x2-9]
        Jx=[2x1 2x2 ; x2 x1] 
    elseif pnum==9
        fx=sin(π*x1/12)cos(π*x2/16)
        gx=[(π/12)cos(π*x1/12)cos(π*x2/16);-(π/16)sin(π*x1/12)sin(π*x2/16)]
        Hx=[(-((π/12)^2)*sin(π*x1/12)*cos(π*x2/16)) (-(π/12)*(π/16)*cos(π*x1/12)*sin(π*x2/16));
            (-(π/12)*(π/16)*cos(π*x1/12)*sin(π*x2/16)) (-((π/16)^2)*sin(π*x1/12)*cos(π*x2/16)) ]
        cx=4x1-3x2
        Jx=[4 -3] 
    end
    return(fx,gx,Hx,cx,Jx)
end

hockobj (generic function with 2 methods)

In [10]:
function lagrangefunc(x, y, ρ, pnum)
    (fx,gx,Hx,cx,Jx)=hockobj(x, pnum)
    Lp = fx -y'cx + (ρ/2)dot(cx,cx)
    ∇Lp = gx - Jx'y + ρ*Jx'cx
    
    Lp=Lp[1]
    ∇Lp=[∇Lp[1];∇Lp[2]]
    
    return(Lp, ∇Lp)
end

lagrangefunc (generic function with 1 method)

In [11]:
function auglagrangian( obj, x0; pnum=6, maxIts=100, relTol=1e-6, absTol=1e-2)
    
    # Initialize
    k=0
    ηk=.10
    x=x0
    (~,~,~,cx,~)=obj(x, pnum)
    y=0.*cx
    ρ=0.00001
    
    # Perform Initial Step
    (x)=penalizedbfgs(lagrangefunc,x0,y,ρ,pnum=pnum)
    (fx,gx,~,cx,Jx)=hockobj(x, pnum)
    k=k+1
    
    # Begin Loop
    while ( norm(cx)>1e-6 || norm(gx-Jx'y)>1e-6 ) && (k <= maxIts)
        if norm(cx)<ηk
            y=y-ρ*cx
            ηk=ηk/2
        else
            ρ=5*ρ
        end
        (x)=penalizedbfgs(lagrangefunc,x,y,ρ,pnum=pnum)
        (fx,gx,~,cx,Jx)=hockobj(x, pnum)
        k=k+1
    end
    
    return(x, fx, norm(gx), norm(cx), k)
end

auglagrangian (generic function with 1 method)

In [12]:
function penalizedbfgs( obj, x0, yl, ρ; pnum=6, maxIts=100, relTol=1e-4)
    
    # Initialize Process
    its=0
    (f, g) = obj(x0,yl,ρ,pnum)
    g0=g
    x=x0
    
    # Initialize with Gradient Descent
    Binv=eye(length(g))                     # Start with B=I
    d=-Binv*g                               # Determine step direction
    (alfa)=wolfesearch(x,d,yl,ρ,obj,pnum)     # Backtracking linesearch
    x=x+alfa*d                              # Take a step
    its=its+1                               # Increase step counter
    gold=g                                  # Save previous gradient
    (f, g) = obj(x,yl,ρ,pnum)                 # Evaluate function for new x

    # Begin Minimization
    while (norm(g) > (norm(g0)*relTol)) && (its <= maxIts)
        s=alfa*d                            # Compute s
        y=g-gold                            # Compute y
        Binv=Binv+(dot(s,y)+dot(y,Binv*y))*(s*s')/dot(s,y)^2-(Binv*y*s'+s*y'*Binv)/dot(s,y) # Determine B^-1
        d=-Binv*g
        alfa=wolfesearch(x,d,yl,ρ,obj,pnum)     # Backtracking linesearch
        xold=x
        x=x+alfa*d                          # Take a step
        its=its+1                           # Increase step counter
        gold=g                              # Save previous gradient
        (f, g) = obj(x,yl,ρ,pnum)             # Evaluate function for new x 
    end
    
    return(x)
end

penalizedbfgs (generic function with 1 method)

In [24]:
# Forward/backward linesearch (Wolfe)
function wolfesearch(x,d,y,ρ,obj,pnum) 
    # Armijo condition linesearch
    mu=1e-4
    alfa_ar=1
    (f0, g0) = obj(x,y,ρ,pnum)
    (fn, gn) = obj(x+alfa_ar*d,y,ρ,pnum)
    k=1
    while fn > f0 + alfa_ar*mu*dot(g0,d)
        alfa_ar=0.5*alfa_ar
        (fn,~)=obj(x+alfa_ar*d,y,ρ,pnum)
    end 
    # Wolfe condition linesearch
    k=1
    maxits=25                                           # Maximum iterations
    eta=[logspace(-3,0,maxits);1]                       # W.C. bound: relax as k->maxits
    alfa_min=0                                          # Initial lower bound on alfa
    alfa_max=alfa_ar/0.5                                # Initial upper bound on alfa
    
    alfa=(alfa_min+alfa_max)/2                          # Initial guess for alfa
    alfa_lo=(3*alfa_min+alfa)/4
    alfa_hi=(alfa+3*alfa_max)/4
    
    (fmin,~) = obj(x+alfa_min*d,y,ρ,pnum)               #                       
    (f,g)    = obj(x+alfa*d,y,ρ,pnum)                   # Evaluate at the guess and at each bound.  
    (fmax,~) = obj(x+alfa_max*d,y,ρ,pnum)               #
    
    while abs(dot(g,d)/dot(g0,d))>eta[k] && k<=maxits
        if f>=fmax && f>=fmin                           # If the function value is greater at the guess than at 
            alfa=(alfa_max-alfa_min)*rand(1)+alfa_min   # either bound, make a new guess.
        else
            if f < fmax && f <fmin                      #
                if fmin>fmax                            #
                    alfa_max=alfa_hi                    #
                else                                    #
                    alfa_min=alfa_lo                    # 
                end                                     # Redefine bounds based on function values
            elseif fmin < f && f < fmax                 #
                alfa_max=alfa_hi                        #
            elseif fmax < f && f < fmin                 #
                alfa_min=alfa_lo                        #
            end                                         
            alfa=(alfa_min+alfa_max)/2                  # Make a guess between new bounds.
        end

        alfa_lo=(3*alfa_min+alfa)/4
        alfa_hi=(alfa+3*alfa_max)/4
    
        alfa_min=alfa_min[1]
        alfa=alfa[1]
        alfa_max=alfa_max[1]
        
        (fmin,~) = obj(x+alfa_min*d,y,ρ,pnum)           #                       
        (f,g)    = obj(x+alfa*d,y,ρ,pnum)               # Evaluate at the guess and at each bound.
        (fmax,~) = obj(x+alfa_max*d,y,ρ,pnum)           #
        k=k+1
    end
    
    return(alfa)
end

wolfesearch (generic function with 1 method)

In [25]:
# Define problem to solve, call solver
pnum=9

x0=[-1.2;1]*(pnum==6)+[2;2]*(pnum==7)+[2;1]*(pnum==8)+[0;0]*(pnum==9)

(xs, fs, gs, cs, k)=auglagrangian(hockobj,x0,pnum=pnum,maxIts=100,relTol=1e-6,absTol=1e-2)

println("x*=$xs")
println("f*=$fs")
println("g*=$gs")
println("c*=$cs")
println("its=$k")

x*=[-27.000000000009265,-36.000000000016186]
f*=-0.49999999999962425
g*=0.16362461737421163
c*=1.1496581464598421e-11
its=11
